In [16]:
import re
from functools import cache
from typing import NamedTuple
from aoc import submit

DAY = 22

In [17]:
class Interval(NamedTuple):
    start: int
    stop: int

    def __bool__(self):
        return self.length > 0

    @property
    def length(self):
        return max(0, self.stop - self.start + 1)

    def intersects(self, other):
        return not (self.start > other.stop or self.stop < other.start)

    def intersection(self, other):
        return Interval(max(self.start, other.start), min(self.stop, other.stop))

    def clip(self, mi, mx):
        start, stop = self
        return Interval(max(mi, start), min(mx, stop))

    @staticmethod
    def parse(line: str):
        nums = map(int, re.findall(r"(-?\d+)", line))
        return map(Interval, nums, nums)


class Cuboid(NamedTuple):
    height: Interval
    width: Interval
    depth: Interval

    @property
    def volume(self):
        return self.height.length * self.width.length * self.depth.length

    def intersects(self, other):
        return (
                self.height.intersects(other.height)
                and self.width.intersects(other.width)
                and self.depth.intersects(other.depth)
        )

    def intersect(self, other):
        return Cuboid(
            *(i.intersection(j) for i, j in zip(self, other))
        )

    def clip(self, mi, mx):
        h, w, d = self
        return Cuboid(h.clip(mi, mx), w.clip(mi, mx), d.clip(mi, mx))

    def __sub__(self: 'Cuboid', other: 'Cuboid'):
        if not self.intersects(other):
            yield self
            return

        intersection = self.intersect(other)

        if i := Interval(self.depth.start, intersection.depth.start - 1):
            yield Cuboid(self.height, self.width, i)

        if i := Interval(intersection.depth.stop + 1, self.depth.stop):
            yield Cuboid(self.height, self.width, i)

        if i := Interval(self.height.start, intersection.height.start - 1):
            yield Cuboid(i, self.width, intersection.depth)

        if i := Interval(intersection.height.stop + 1, self.height.stop):
            yield Cuboid(i, self.width, intersection.depth)

        if i := Interval(self.width.start, intersection.width.start - 1):
            yield Cuboid(intersection.height, i, intersection.depth)

        if i := Interval(intersection.width.stop + 1, self.width.stop):
            yield Cuboid(intersection.height, i, intersection.depth)


def regions():
    """
    Split intersecting cubes into individual regions.
    """


@cache
def parse_input(raw):
    instructions = [(line[:2] == 'on', Cuboid(*Interval.parse(line)))
                    for line in raw.splitlines()]

    cuboids = []
    for is_on, current in instructions:
        diffs = []
        for cuboid in cuboids:
            diffs.extend(cuboid - current)
        if is_on:
            diffs.append(current)
        cuboids = diffs
    return cuboids


@submit(day=DAY)
def part_one(raw):
    cuboids = parse_input(raw)
    return sum(cuboid.clip(-50, 50).volume for cuboid in cuboids)

part_one:
✅ example: 590784         (3.69 ms)
✅ input:   533863         (350.97 ms)


In [18]:
@submit(day=DAY, skip_example=True)
def part_two(raw):
    cuboids = parse_input(raw)
    return sum(cuboid.volume for cuboid in cuboids)

part_two:
✅ input:   1261885414840992 (4.95 ms)
